In [1]:
# Translated to .py by Meritxell Pacheco (December 2016)
# Adapted to PandasBiogeme by Nicola Ortelli (November 2019)

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.optimization import newtonTrustRegionForBiogeme
from biogeme.expressions import Beta, DefineVariable, bioDraws, MonteCarlo
from biogeme.models import loglogit, logit, log

df = pd.read_csv("swissmetro.dat", sep = '\t')
database = db.Database("swissmetro", df)

globals().update(database.variables)

# Exclude data
exclude = (( PURPOSE != 1 ) * ( PURPOSE != 3 ) + ( CHOICE == 0 )) > 0
database.remove(exclude)

In [2]:
# Parameters to be estimated
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 1)
ASC_SM    = Beta('ASC_SM',    0, None, None, 0)
ASC_CAR   = Beta('ASC_CAR',   0, None, None, 0)

B_TIME = Beta('B_TIME', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)
B_HE   = Beta('B_HE',   0, None, None, 0)

M_RAIL = Beta('M_RAIL', 0, None, None, 1)
S_RAIL = Beta('S_RAIL', 1, None, None, 0)
M_EX = Beta('M_EX', 0, None, None, 1)
S_EX = Beta('S_EX', 1, None, None, 0)
XI_RAIL = M_RAIL + S_RAIL * bioDraws('XI_RAIL', 'NORMAL')
XI_EX = M_EX + S_EX * bioDraws('XI_EX', 'NORMAL')

# Definition of new variables
TRAIN_COST = DefineVariable('TRAIN_COST', TRAIN_CO * ( GA == 0 ), database)
SM_COST    = DefineVariable('SM_COST', SM_CO * ( GA == 0 ), database)

# Utilities
V_TRAIN = ASC_TRAIN + B_TIME * TRAIN_TT + B_COST * TRAIN_COST + B_HE * TRAIN_HE + XI_RAIL + XI_EX
V_SM    = ASC_SM    + B_TIME * SM_TT    + B_COST * SM_COST    + B_HE * SM_HE + XI_RAIL
V_CAR   = ASC_CAR   + B_TIME * CAR_TT   + B_COST * CAR_CO + XI_EX

V = {1: V_TRAIN, 2: V_SM, 3: V_CAR}
av = {1: TRAIN_AV, 2: SM_AV, 3: CAR_AV}

# Choice model estimation
prob = logit(V, av, CHOICE)
logprob = log(MonteCarlo(prob))
biogeme = bio.BIOGEME(database, logprob, numberOfDraws = 100)
biogeme.modelName = "Error Component 02"
results = biogeme.estimate(algorithm = newtonTrustRegionForBiogeme)

# Results
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) = {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

            Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR  0.247579  0.102189   2.422770  1.540267e-02      0.108683   
ASC_SM   0.852328  0.120685   7.062400  1.636469e-12      0.210617   
B_COST  -0.016872  0.001017 -16.589065  0.000000e+00      0.001601   
B_HE    -0.007659  0.001359  -5.637094  1.729434e-08      0.001343   
B_TIME  -0.016457  0.000906 -18.160266  0.000000e+00      0.001931   
S_EX     2.939391  0.268978  10.927995  0.000000e+00      0.468859   
S_RAIL   0.003125  0.182606   0.017112  9.863476e-01      0.095235   

         Rob. t-test  Rob. p-value  
ASC_CAR     2.278001  2.272651e-02  
ASC_SM      4.046815  5.191932e-05  
B_COST    -10.539375  0.000000e+00  
B_HE       -5.701438  1.188007e-08  
B_TIME     -8.522308  0.000000e+00  
S_EX        6.269249  3.627931e-10  
S_RAIL      0.032810  9.738261e-01  
Nbr of observations: 6768
LL(0) = -6830.469
LL(beta) = -5262.574
rho bar square = 0.229
Output file: Error Component 02.html
